In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Pancreatic_Cancer"
cohort = "GSE223409"

# Input paths
in_trait_dir = "../../input/GEO/Pancreatic_Cancer"
in_cohort_dir = "../../input/GEO/Pancreatic_Cancer/GSE223409"

# Output paths
out_data_file = "../../output/preprocess/Pancreatic_Cancer/GSE223409.csv"
out_gene_data_file = "../../output/preprocess/Pancreatic_Cancer/gene_data/GSE223409.csv"
out_clinical_data_file = "../../output/preprocess/Pancreatic_Cancer/clinical_data/GSE223409.csv"
json_path = "../../output/preprocess/Pancreatic_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Dual targeted extracellular vesicles regulating oncogenic genes in pancreatic cancer"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['cell type: Mouse Embryonic Fibroblast (MEF)', 'cell type: Bone Marrow Stem Cells (BMSCs)'], 1: ['tissue: extracellular vesicles (EVs)'], 2: ['treatment: Plasmids cd64_TP53 treated', 'treatment: PBS treated', 'treatment: Control']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# Analysis of dataset GSE223409 for Pancreatic Cancer study

# 1. Gene Expression Data Availability
# Based on the background information, this seems to be about EVs and gene expression
# The series contains information about extracellular vesicles and TP53 gene regulation
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# Looking at the sample characteristics dictionary:

# 2.1 Trait (Pancreatic Cancer)
# There is no direct mention of pancreatic cancer status in the sample characteristics
# This appears to be an experimental dataset with different treatments rather than patient samples
trait_row = None  # No trait information available

# Since we don't have trait information, we'll define a placeholder function
def convert_trait(value):
    return None

# 2.2 Age
# No age information is provided in the sample characteristics
age_row = None  # No age information available

def convert_age(value):
    return None

# 2.3 Gender
# No gender information is provided in the sample characteristics
gender_row = None  # No gender information available

def convert_gender(value):
    return None

# 3. Save Metadata
# The dataset doesn't contain the clinical trait information we need (is_trait_available = False)
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is None, we skip this substep
# No need to extract clinical features when trait data is not available


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
       '14', '15', '16', '17', '18', '19', '20'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# The identifiers in the gene expression data are numeric (1, 2, 3, etc.) and not standard human gene symbols
# These appear to be arbitrary numeric identifiers and would need to be mapped to actual gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1', '2', '3', '4', '5'], 'COL': ['192', '192', '192', '192', '192'], 'ROW': [328.0, 326.0, 324.0, 322.0, 320.0], 'NAME': [nan, nan, nan, 'A_55_P2051983', 'A_52_P169082'], 'SPOT_ID': ['GE_BrightCorner', 'DarkCorner', 'DarkCorner', nan, nan], 'CONTROL_TYPE': ['pos', 'pos', 'pos', 'FALSE', 'FALSE'], 'GB_ACC': [nan, nan, nan, 'NM_001001803', 'NM_021294'], 'GENE_ID': [nan, nan, nan, 408198.0, 13168.0], 'GENE_SYMBOL': [nan, nan, nan, 'Spink7', 'Dbil5'], 'GENE_NAME': [nan, nan, nan, 'serine peptidase inhibitor, Kazal type 7 (putative)', 'diazepam binding inhibitor-like 5'], 'UNIGENE_ID': [nan, nan, nan, 'Mm.478742', 'Mm.347413'], 'ENSEMBL_ID': [nan, nan, nan, 'ENSMUST00000076194', nan], 'ACCESSION_STRING': [nan, nan, nan, 'ref|NM_001001803|ens|ENSMUST00000076194|gb|BC148575|gb|BC156750', 'ref|NM_021294'], 'CHROMOSOMAL_LOCATION': [nan, nan, nan, 'chr18:62753954-62753895', 'chr11:76031955-76032014'], 'CYTOBAND': [nan, nan, nan, 'mm|18qE1', 'mm|11qB5'], 'DESCRIP

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify which columns in gene_annotation store identifiers and gene symbols
# From the preview, we can see that gene identifiers are in 'ID' column which match the gene expression data indices
# Gene symbols are in the 'GENE_SYMBOL' column

# 2. Create a gene mapping dataframe with the identifier and gene symbol columns
gene_mapping = get_gene_mapping(gene_annotation, 'ID', 'GENE_SYMBOL')

# 3. Convert probe-level measurements to gene expression data
# This applies the gene mapping to the gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Let's preview the resulting gene expression data
print("Gene expression data after mapping (first 5 rows):")
print(preview_df(gene_data))


Gene expression data after mapping (first 5 rows):
{'GSM6947333': [5.765499247, 8.90549615, 6.351152373, 5.722009762, 5.854746855], 'GSM6947334': [5.733167493, 8.309656336, 5.996497708, 6.470031597, 5.814009788], 'GSM6947335': [5.700167959, 5.951790839, 5.742400314, 6.012391263, 6.364721615], 'GSM6947336': [5.891782887, 5.840510505, 5.650289558, 6.008373763, 6.932568617], 'GSM6947337': [5.747773928, 8.780743481, 6.886196389, 6.059094686, 5.963228122], 'GSM6947338': [5.929038451, 6.019699325, 5.8293881, 5.818277684, 6.383367386], 'GSM6947339': [5.935432529, 5.968244709, 5.765170731, 5.890302204, 6.405747256], 'GSM6947340': [5.663317403, 5.782679504, 5.686461768, 5.826241194, 7.061529204], 'GSM6947341': [5.699334692, 5.967858065, 5.799722935, 6.18135465, 5.997149675], 'GSM6947342': [5.88262595, 7.252686231, 8.345568571, 5.797517166, 5.602564453], 'GSM6947343': [5.875824971, 6.02315016, 5.696309368, 5.997787261, 6.10026366], 'GSM6947344': [5.648246035, 7.499101979, 8.462235322, 5.85624817

### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Based on our analysis in Step 2, we determined that trait data is not available
# So we don't need to extract clinical features or create linked_data

# Create a minimal DataFrame to satisfy the requirements of validate_and_save_cohort_info
# We'll use the first few rows of our normalized gene data to provide structure
df_for_validation = pd.DataFrame(index=normalized_gene_data.index[:5], columns=normalized_gene_data.columns[:5])

# 5. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=False,  # We determined this in Step 2
    is_biased=True,  # Set to True since absence of trait data means we can't use it for analysis
    df=df_for_validation,
    note="Dataset contains gene expression data but lacks pancreatic cancer trait information required for analysis."
)

# 6. Since trait data is not available, this dataset is not usable for our trait analysis
# We've saved the gene expression data, but we won't create or save linked data
print("Data quality check failed due to missing trait information. Only gene data saved.")

Normalized gene data saved to ../../output/preprocess/Pancreatic_Cancer/gene_data/GSE223409.csv
Data quality check failed due to missing trait information. Only gene data saved.
